In [1]:
from pyarrow import feather
from pathlib import Path
import requests
from cairosvg import svg2png
from io import BytesIO
from tqdm import tqdm, trange
import re
import pdb
import pandas as pd
import json
import pickle
from PIL import Image

In [14]:
data_path = Path('/data/kaiwen/ahrefs/dataset')

train_df = feather.read_feather('/data/kaiwen/ahrefs/dataset/train_split.feather')
val_df = feather.read_feather('/data/kaiwen/ahrefs/dataset/val_split.feather')

In [16]:
train_df.loc[train_df['id']=='dbccc72ea88ef231a123ce9c08745244']['src'].values[0]

'https://cdn.dealerspike.com/imglib/v1/800x600/imglib/Assets/Inventory/F4/97/F497DB03-F9FE-4DBF-AB2B-513C3EE1C700.jpg'

In [11]:
with open("/data/kaiwen/ahrefs/dataset/lost_relevant.json", 'r') as f:
    d = json.load(f)
    
len(d)

5664

In [9]:
p = Path('/data/kaiwen/ahrefs/dataset/train_data')
len(list(p.rglob("*.pkl")))

48910

In [17]:
dfs = [train_df, val_df]
names = ['train', 'val']

failed = []
df_save = []
# some images are blank, which will cause error when fine-tuning
corrupted = ['af408364133664b68836a9a474717ce5', 'dbccc72ea88ef231a123ce9c08745244']

for name, df in zip(names, dfs):
    data_path_split = data_path / f"{name}_data"
    for i in range(len(df)):
        file_id = df['id'][i]
        is_relevant = df['is_relevant'][i]
        query = df['query'][i]
        folder_num = i // 1000
        data_path_final = data_path_split / f'{(folder_num+1)*1000}/{file_id}.pkl'
        if df['is_relevant'][i]==1 and data_path_final.is_file() and file_id not in corrupted:
            with open(data_path_final, 'rb') as f:
                img = pickle.load(f)['image']
            try:
                image = Image.open(BytesIO(img)).convert("RGB")
                row = {'id': file_id, 'query': query, 'img_path': str(data_path_final)}
                df_save.append(row)
            except:
                failed.append(file_id)
                continue

df_save = pd.DataFrame(df_save).reset_index(drop=True)
print(len(df_save))
df_save.to_feather(data_path / "train_final_df.feather")

/data/conda/envs/layoutlmv3/lib/python3.7/site-packages/PIL/Image.py:997: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "
/data/conda/envs/layoutlmv3/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:858: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 2. 
  warnings.warn(str(msg))
/data/conda/envs/layoutlmv3/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:858: UserWarning: Corrupt EXIF data.  Expecting to read 2 bytes but only got 0. 
  warnings.warn(str(msg))


15336


In [11]:
15336

209

In [27]:
d1 = feather.read_feather('/data/kaiwen/ahrefs/dataset/train_final_df.feather')
d2 = feather.read_feather('/data/kaiwen/ahrefs/dataset/val_final_df.feather')

len(list(set(d1['query'].unique()).union(set(d2['query'].unique()))))
# list(set(d1['query'].unique()).update(set(d2['query'].unique())))

730

In [29]:
labels = [1, 2, 3]
s = [0 for l in labels if l==1 else 1]

SyntaxError: invalid syntax (3977427914.py, line 2)

In [30]:
to_skip = []
img_error = []

In [31]:
# user_agent='Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
user_agent = 'Mozilla/5.0 (Linux; U; Android 4.0.4; en-gb; GT-I9300 Build/IMM76D) AppleWebKit/534.30 (KHTML, like Gecko) Version/4.0 Mobile Safari/534.30'
requests.adapters.DEFAULT_RETRIES = 3

for i in trange(1000):
    id = df['id'][i]
    img_url = df['src'][i]
    parsed_url = re.search(r'(.*\.(jpg|JPG|png|PNG|svg|npj|webp|gif))(.*)', img_url)
    img_url = parsed_url.group(1) if parsed_url else img_url
    if (data_path / f"1000/{id}.pkl").is_file() or id in to_skip:
        continue
    
    surfix = img_url.split(".")[-1].split("?")[0]
    try:
        response = requests.get(img_url, headers={"User-Agent": user_agent}, timeout=10, stream=True)
    except:
        to_skip.append(id)
        continue
    if response.status_code==200:
        try:
            if surfix=='svg':
                img = svg2png(bytestring=BytesIO(response.content).read())
            else:
                img = BytesIO(response.content).getvalue()
        except:
            print(img_url)
            img_error.append(img_url)
    else:
        to_skip.append(id)
        continue

  8%|▊         | 85/1000 [01:29<18:15,  1.20s/it]

In [20]:
import requests
requests.adapters.DEFAULT_RETRIES = 5

img_url = 'https://cdn.myoperator.com/img/partner/samsung.svg'
parsed_url = re.search(r'(.*\.(jpg|JPG|png|PNG|svg|npj|webp|gif))(.*)', img_url)
img_url = parsed_url.group(1) if parsed_url else img_url
print(img_url)
# session = requests.Session()
# retry = Retry(connect=3, backoff_factor=0.5)
# adapter = HTTPAdapter(max_retries=retry)
# session.mount('http://', adapter)
# session.mount('https://', adapter)

# session.get(img_url)

# img_url = re.search(r'(.*\.(jpg|JPG|png|PNG|svg|npj|webp|gif))(.*)', img_url).group(1)
response = requests.get(img_url, verify=False, timeout=10, stream=True)
response.status_code

https://cdn.myoperator.com/img/partner/samsung.svg


/data/conda/envs/layoutlmv3/lib/python3.7/site-packages/urllib3/connectionpool.py:1052: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cdn.myoperator.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


200